In [1]:
# Convert md -> file


Target: Convert markdown file -> vector database

paper:{meta-json} --- pdf file_id --- paper md --- vector embedding

In [2]:
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from transformers import logging

# logging.set_verbosity_warning()

logging.set_verbosity_error()

sentences = ['That is a happy person', 'That is a very happy person']
model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
embeddings = model.encode(sentences)
print(cos_sim(embeddings[0], embeddings[1]))

/home/chivier/opt/miniconda3/envs/darxiv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange
/home/chivier/opt/miniconda3/envs/darxiv/lib/python3.10/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/chivier/opt/miniconda3/envs/darxiv/lib/python3.10/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
/home/chivier/opt/miniconda3/envs/darxiv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download`

tensor([[0.9769]])


In [3]:
md_path = "../data/ocr/ocr_2024-06-03/2402-04264_1/2402-04264_1.md"
from transformers import AutoModel, AutoTokenizer
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
from langchain.text_splitter import RecursiveCharacterTextSplitter

def get_embeddings(sentences):
    model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
    embeddings = model.encode(sentences)
    return embeddings

def convert_markdown_to_chunks(md_file, chunk_size=8000, overlap=100):
    separators = ['\\n\\n', '\\n', ' ']
    splitter = RecursiveCharacterTextSplitter(separators, chunk_size, overlap)
    with open(md_file, 'r') as f:
        text = f.read()
    chunks = splitter.split_text(text)
    return chunks
    

chunks = convert_markdown_to_chunks(md_path)

In [4]:
print(chunks)

['# Analysis Of Hopfield Model As Associative Memory Silvestri Matteo\n\nDepartment of Mathematics, University of Rome "La Sapienza"\n\n## Abstract\n\nThis article delves into the Hopfield neural network model, drawing inspiration from biological neural systems. The exploration begins with an overview of the model\'s foundations, incorporating insights from mechanical statistics to deepen our understanding.\n\nFocusing on *audio retrieval*, the study demonstrates the Hopfield model\'s associative memory capabilities. Through practical implementation, the network is trained to retrieve different patterns.\n\n## 1 An Overview Of Neuroscience\n\nThe Hopfield model finds inspiration in the intricate connections among biological neurons in the human brain, mimicking nature\'s efficiency in information processing. Similar to the synaptic communication in biological systems, the Hopfield network utilizes connections to store and retrieve patterns. Before delving further, we\'ll provide a brie

In [5]:
print([len(chunk) for chunk in chunks])

[3846, 3678, 3961, 3779, 3915, 3916, 3840, 3925, 3982, 3655, 3830, 3948, 3574, 3841, 3811, 3985, 3987, 3097, 3458, 3593, 3568]


# Store into index_format (json temperary version 1)



In [13]:
import os
import csv
import torch
import json
import numpy as np

def create_vecindex(source_path, index_path, chunk_size=8000, overlap=100):
    # Search for markdown files in source_path recursively
    markdown_files = []
    for root, dirs, files in os.walk(source_path):
        for file in files:
            if file.endswith('.md'):
                markdown_files.append(os.path.join(root, file))
    print(markdown_files)
    
    model = SentenceTransformer('Alibaba-NLP/gte-large-en-v1.5', trust_remote_code=True)
    
    if not os.path.exists(index_path):
        os.makedirs(index_path)
        
    for md_file in markdown_files:
        chunks = convert_markdown_to_chunks(md_file, chunk_size, overlap)
        pure_filename = os.path.splitext(os.path.basename(md_file))[0]
        chunk_filename = os.path.join(index_path, pure_filename + '_chunks.csv')
        vector_filename = os.path.join(index_path, pure_filename + '_vectors.npy')
        embeddings = []
        for chunk_id in range(0, len((chunks)), 8):
            embeddings.extend(model.encode(chunks[chunk_id:chunk_id+8]))
        with open(chunk_filename, 'w', newline='') as csvfile:
            fieldnames = ['id', 'chunk']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

            writer.writeheader()

            chunk_format = lambda string: repr(string)

            for i in range(len(chunks)):
                writer.writerow({'id': i, 'chunk': chunk_format(chunks[i])})
        np.save(vector_filename, embeddings)
        
        # store chunks into 
        torch.cuda.empty_cache() 
        
    return csv_result

    with open(index_file, 'w', newline='') as csvfile:
        fieldnames = ['id', 'chunk', 'embedding']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()

        for md_file in markdown_files:
            chunks = convert_markdown_to_chunks(md_file, chunk_size, overlap)
            embeddings = get_embeddings(chunks)

            for i in range(len(chunks)):
                writer.writerow({'id': id, 'chunk': chunks[i], 'embedding': embeddings[i]})
                id += 1


In [14]:
result = create_vecindex("../data/ocr/ocr_2024-06-03", "../data/index/index_2024-06-03")

['../data/ocr/ocr_2024-06-03/2405-20467_1/2405-20467_1.md', '../data/ocr/ocr_2024-06-03/2405-20662_1/2405-20662_1.md', '../data/ocr/ocr_2024-06-03/2405-20715_1/2405-20715_1.md', '../data/ocr/ocr_2024-06-03/2405-20750_1/2405-20750_1.md', '../data/ocr/ocr_2024-06-03/2405-20630_1/2405-20630_1.md', '../data/ocr/ocr_2024-06-03/2405-20525_1/2405-20525_1.md', '../data/ocr/ocr_2024-06-03/2405-21009_1/2405-21009_1.md', '../data/ocr/ocr_2024-06-03/2405-21051_1/2405-21051_1.md', '../data/ocr/ocr_2024-06-03/2405-20559_1/2405-20559_1.md', '../data/ocr/ocr_2024-06-03/2405-20524_1/2405-20524_1.md', '../data/ocr/ocr_2024-06-03/2405-21040_1/2405-21040_1.md', '../data/ocr/ocr_2024-06-03/2405-20423_1/2405-20423_1.md', '../data/ocr/ocr_2024-06-03/2405-20456_1/2405-20456_1.md', '../data/ocr/ocr_2024-06-03/2405-21066_1/2405-21066_1.md', '../data/ocr/ocr_2024-06-03/2405-20502_1/2405-20502_1.md', '../data/ocr/ocr_2024-06-03/2405-20603_1/2405-20603_1.md', '../data/ocr/ocr_2024-06-03/2405-20980_1/2405-20980_1.m

/home/chivier/opt/miniconda3/envs/darxiv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


NameError: name 'csv_result' is not defined

In [ ]:
import torch
torch.cuda.empty_cache() 

In [12]:
string_with_newline = "Here is a string with a newline character: \n"
escaped_string = string_with_newline.encode('unicode_escape').decode('utf-8')
print(escaped_string)


Here is a string with a newline character: \n


In [11]:
print(r"Here is a string with a newline character: \n")


Here is a string with a newline character: \n
